# CassIO, mini demo

In [1]:
import os
import cassio

## Initialize

In [2]:
token = os.environ['ASTRA_DB_APPLICATION_TOKEN']
database_id = os.environ['ASTRA_DB_DATABASE_ID']

#print(f"OPENAI_API_KEY             = {os.environ['OPENAI_API_KEY'][:6]} ...")
print(f"token       = {token[:12]} ...")
print(f"database_id = {database_id[:4]} ...")

token       = AstraCS:fZuF ...
database_id = 4f83 ...


In [3]:
cassio.init(token=token, database_id=database_id)

In [4]:
from langchain.embeddings import OpenAIEmbeddings
emb_fun = OpenAIEmbeddings()

## CassIO layer

### Create a vector store and put a couple of items

In [5]:
v_store = cassio.table.VectorCassandraTable(table="demo_v_store", vector_dimension=1536)

In [6]:
docs = [
    "The cat is on the table",
    "Please store the appliance safely when not using it",
    "All happy families are alike, each unhappy family is unhappy in its own way",
]

In [7]:
for i, txt in enumerate(docs):
    v_store.put(row_id=f"row{i}", body_blob=txt, vector=emb_fun.embed_query(txt))
    print(f"Inserted '{txt}'")

Inserted 'The cat is on the table'
Inserted 'Please store the appliance safely when not using it'
Inserted 'All happy families are alike, each unhappy family is unhappy in its own way'


### Run an ANN search

In [8]:
query = "Is a feline found around here?"

results = v_store.ann_search(n=1, vector=emb_fun.embed_query(query))

print(f"Query: '{query}'")
print("Result(s):")
for r in results:
    print(f"    [{r['row_id']}]: '{r['body_blob']}'")

Query: 'Is a feline found around here?'
Result(s):
    [row0]: 'The cat is on the table'


## LangChain layer

In [9]:
from langchain.llms import OpenAI

llm = OpenAI()

### Set a cache for LLMs

In [10]:
from langchain.cache import CassandraCache  # CassandraSemanticCache available as well...

import langchain
langchain.llm_cache = CassandraCache()

#### First time

In [11]:
%%time
print(llm("What does 'All Hands' mean?").strip() + "\n")

In a military or nautical context, “all hands” is a call to all personnel to attend a meeting or to perform a task. It is a term used to summon the entire crew to assemble on deck.

CPU times: user 12.5 ms, sys: 239 µs, total: 12.8 ms
Wall time: 1.68 s


#### Again...

In [12]:
%%time
print(llm("What does 'All Hands' mean?").strip() + "\n")

In a military or nautical context, “all hands” is a call to all personnel to attend a meeting or to perform a task. It is a term used to summon the entire crew to assemble on deck.

CPU times: user 3.67 ms, sys: 0 ns, total: 3.67 ms
Wall time: 119 ms


## (cleanup)

In [13]:
v_store.clear()

In [14]:
langchain.llm_cache.clear()